In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Setup browser
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# Go to TradingView
url = "https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/"
driver.get(url)

# Wait for the table to load
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
print("✅ Table loaded")

# Load stocks until at least 1000 rows are visible
def load_1000_stocks():
    while True:
        rows = driver.find_elements(By.XPATH, '//table//tbody/tr')
        print(f"📊 Rows loaded: {len(rows)}")
        if len(rows) >= 1000:
            break
        try:
            load_more_btn = driver.find_element(By.XPATH, '//button[.//span[text()="Load More"]]')
            load_more_btn.click()
            time.sleep(2)
        except:
            print("❌ 'Load More' not found or failed. Ending early.")
            break

load_1000_stocks()
time.sleep(2)

# ✅ Extract data from visible table using Selenium directly
rows = driver.find_elements(By.XPATH, '//table//tbody/tr')
data = []

for row in rows[:1000]:  # Only first 1000 rows
    try:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if len(cells) < 6:
            continue

        symbol_name = cells[0].text.split('\n')
        symbol = symbol_name[0] if len(symbol_name) > 0 else ""
        name = symbol_name[1] if len(symbol_name) > 1 else ""

        last_price = cells[2].text.strip()
        change_percent = cells[3].text.strip()
        volume = cells[4].text.strip()
        market_cap = cells[5].text.strip()

        data.append({
            "Symbol": symbol,
            "Name": name,
            "Last Price": last_price,
            "Change %": change_percent,
            "Volume": volume,
            "Market Cap": market_cap
        })
    except Exception as e:
        print(f"⚠️ Error reading row: {e}")
        continue

driver.quit()

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("us_1000_stocks.csv", index=False, lineterminator='\n')
print(f"✅ DONE: {len(df)} rows saved to us_1000_stocks.csv")


✅ Table loaded
📊 Rows loaded: 100
📊 Rows loaded: 100
📊 Rows loaded: 200
📊 Rows loaded: 300
📊 Rows loaded: 400
📊 Rows loaded: 500
📊 Rows loaded: 600
📊 Rows loaded: 700
📊 Rows loaded: 800
📊 Rows loaded: 900
📊 Rows loaded: 1000
✅ DONE: 1000 rows saved to us_1000_stocks.csv
